In [ ]:
!pip install datasets

In [ ]:
!pip install unbabel-comet

In [ ]:
!pip install accelerate -U

In [ ]:

!pip install transformers[torch] -U


In [ ]:
import torch
torch.cuda.empty_cache()


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import Softmax

from typing import List, Optional, Tuple, Union, Dict, Any

from datasets import load_dataset, Dataset, DatasetDict, load_metric, load_from_disk
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from transformers import PreTrainedModel, TrainingArguments

from nltk.translate.bleu_score import sentence_bleu
from comet import download_model, load_from_checkpoint

import pandas as pd
import numpy as np

import random
import math
import time
from tqdm import tqdm
import os
import json

# Load Model

In [2]:
model_name = 'm2m100_418M'
experiment = 'en-ha-finetune'
#dataset_name = '/home/spandey7/Language-Adapters/Data/en-ha' #change this to the name of the folder containing the dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model = M2M100ForConditionalGeneration.from_pretrained(f"facebook/{model_name}")
model = model.to(device)
tokenizer = M2M100Tokenizer.from_pretrained(f"facebook/{model_name}")

# Load Data

In [4]:
src_lang = 'ha'
tgt_lang = 'en'
tokenizer.src_lang = "ha"
tokenizer.tgt_lang = "en"

In [5]:
# Load the datasets from Hugging Face Hub
train_dataset = load_dataset("pranjali97/ha-en_RL-grow1_train", split='train')
valid_dataset = load_dataset("pranjali97/ha-en_RL-grow1_valid", split='train')  # Assuming the split is also 'train'


In [ ]:
# Filter the datasets to only include samples with a score > 0.6
train_dataset = train_dataset.filter(lambda example: example['score'] > 0.6)
valid_dataset = valid_dataset.filter(lambda example: example['score'] > 0.6)

In [ ]:
# Define the preprocess function
def preprocess_function(examples):
    inputs = examples['src']  # Hausa sentences
    targets = examples['mt']  # English translations
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    # Return the tokenized inputs and labels
    return {'input_ids': model_inputs['input_ids'], 'attention_mask': model_inputs['attention_mask'], 'labels': labels['input_ids']}

# Apply the preprocess function to the datasets
tokenized_train_dataset = train_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src', 'ref', 'mt', 'score']  # Specify the correct columns to remove
)
tokenized_valid_dataset = valid_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src', 'ref', 'mt', 'score']  # Specify the correct columns to remove
)

# Create the DatasetDict
tokenized_dataset = DatasetDict({
    'train': tokenized_train_dataset,  # Directly assign the processed dataset
    'validation': tokenized_valid_dataset  # Directly assign the processed dataset
})

In [ ]:
tokenized_dataset

##Old Data Processing. Ignore

In [ ]:
old_dataset_name = '/home/spandey7/Language-Adapters/Data/en-ha' #change this to the name of the folder containing the dataset


In [ ]:
old_dataset = DatasetDict({'train':Dataset.from_pandas(pd.read_csv(f'{old_dataset_name}/cleaned_train.csv')),'validation':Dataset.from_pandas(pd.read_csv(f'{old_dataset_name}/cleaned_dev.csv'))})

In [ ]:
old_dataset

In [ ]:
def old_preprocess_function(examples):
    inputs = [example for example in examples[src_lang]]
    targets = [example for example in examples[tgt_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=256, truncation=True, padding="max_length")
    return model_inputs

In [ ]:
old_tokenized_dataset = old_dataset.map(old_preprocess_function, batched=True, remove_columns=old_dataset.column_names['train'])

In [ ]:
old_tokenized_dataset

# Finetune Model

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    eval_accumulation_steps=16,
    fp16=True,
    do_train=True,
    do_eval=True,
    logging_steps=5,
    predict_with_generate=True,
    load_best_model_at_end=True, 
    metric_for_best_model="bleu",  
)


In [ ]:
# Initialize the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Post-process texts
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    # Calculate BLEU scores
    bleu_scores = []
    for pred, label in zip(decoded_preds, decoded_labels):
        # Tokenize the sentences into words
        pred_tokens = pred.split()
        label_tokens = [label.split()]  # BLEU expects a list of possible references
        # Calculate BLEU score for the sentence pair
        # Use a smoothing function to avoid issues with 0 scores for small sentences
        score = sentence_bleu(label_tokens, pred_tokens, smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(score)
    
    # Calculate the average BLEU score
    average_bleu_score = np.mean(bleu_scores)
    
    return {"bleu": average_bleu_score}


In [ ]:
# Initialize the Trainer with the new compute_metrics function
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 
)

In [ ]:
# Add early stopping callback if desired
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))
torch.cuda.empty_cache()
# Train the model
trainer.train()

0.7 Threshold

In [6]:
# Filter the datasets to only include samples with a score > 0.6
train_dataset = train_dataset.filter(lambda example: example['score'] > 0.7)
valid_dataset = valid_dataset.filter(lambda example: example['score'] > 0.7)

Filter:   0%|          | 0/29454 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3339 [00:00<?, ? examples/s]

In [7]:
# Define the preprocess function
def preprocess_function(examples):
    inputs = examples['src']  # Hausa sentences
    targets = examples['mt']  # English translations
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    # Return the tokenized inputs and labels
    return {'input_ids': model_inputs['input_ids'], 'attention_mask': model_inputs['attention_mask'], 'labels': labels['input_ids']}

# Apply the preprocess function to the datasets
tokenized_train_dataset = train_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src', 'ref', 'mt', 'score']  # Specify the correct columns to remove
)
tokenized_valid_dataset = valid_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src', 'ref', 'mt', 'score']  # Specify the correct columns to remove
)

# Create the DatasetDict
tokenized_dataset = DatasetDict({
    'train': tokenized_train_dataset,  # Directly assign the processed dataset
    'validation': tokenized_valid_dataset  # Directly assign the processed dataset
})

Map:   0%|          | 0/802 [00:00<?, ? examples/s]

/home/spandey7/anaconda3/envs/capstone/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/86 [00:00<?, ? examples/s]

In [8]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 802
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 86
    })
})

In [9]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels

In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    eval_accumulation_steps=16,
    fp16=True,
    do_train=True,
    do_eval=True,
    logging_steps=5,
    predict_with_generate=True,
    load_best_model_at_end=True, 
    metric_for_best_model="bleu",  
)


In [11]:
# Initialize the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Post-process texts
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    # Calculate BLEU scores
    bleu_scores = []
    for pred, label in zip(decoded_preds, decoded_labels):
        # Tokenize the sentences into words
        pred_tokens = pred.split()
        label_tokens = [label.split()]  # BLEU expects a list of possible references
        # Calculate BLEU score for the sentence pair
        # Use a smoothing function to avoid issues with 0 scores for small sentences
        score = sentence_bleu(label_tokens, pred_tokens, smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(score)
    
    # Calculate the average BLEU score
    average_bleu_score = np.mean(bleu_scores)
    
    return {"bleu": average_bleu_score}


In [13]:
# Initialize the Trainer with the new compute_metrics function
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
# Add early stopping callback if desired
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))
torch.cuda.empty_cache()
# Train the model
trainer.train()

/home/spandey7/anaconda3/envs/capstone/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu
1,0.403500,0.310282,0.030358
2,0.305400,0.255439,0.000655
3,0.209600,0.251255,0.019529
4,0.124500,0.263800,0.043385
5,0.098900,0.267700,0.036625
6,0.064000,0.280107,0.075176
7,0.058100,0.285017,0.057039
8,0.036800,0.295427,0.075089
9,0.034900,0.292727,0.069664


/home/spandey7/anaconda3/envs/capstone/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/spandey7/anaconda3/envs/capstone/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/spandey7/anaconda3/envs/capstone/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/spandey7/anaconda3/envs/capstone/lib/python3.8/site-packages/torch/nn/parallel/_functions.p

TrainOutput(global_step=306, training_loss=0.36162190700522046, metrics={'train_runtime': 731.296, 'train_samples_per_second': 21.934, 'train_steps_per_second': 0.93, 'total_flos': 3910540254511104.0, 'train_loss': 0.36162190700522046, 'epoch': 9.0})

In [ ]:
#Fine-tuning the model
batch_size = 32
args = Seq2SeqTrainingArguments(
    f"{experiment}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    logging_steps=100,
    logging_dir=f"{experiment}/logs",
    overwrite_output_dir=True,
    save_steps=1000,
    eval_steps=1000,
    warmup_steps=1000,
    save_strategy='epoch'
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    labels = eval_preds.label_ids
    pred_ids = eval_preds.predictions
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]
    
    preds = np.argmax(pred_ids, axis=-1)
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Removeme
    import warnings
    warnings.warn(f"preds: {decoded_preds[0]}\n)")
    warnings.warn(f"labels: {decoded_labels[0]}\n)")

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
training_args = TrainingArguments(
    f"./base_finetune/{experiment}/model",
    # evaluation_strategy="steps",
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    warmup_steps=1000,
    # lr_scheduler_type='constant',
    # gradient_accumulation_steps=4,
    eval_accumulation_steps=16,
    # gradient_checkpointing=True,
    # predict_with_generate=True,
    fp16=True,
    do_train=True,
    do_eval=True,
    logging_steps=5,
    # eval_steps=5,
    save_strategy="epoch",
    metric_for_best_model="bleu",
    load_best_model_at_end=True,
)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

# Grow Step

In [ ]:
# Grow function
def grow(model, input_texts, num_samples=5):
    generated_texts = []
    for text in input_texts:
        # Generate multiple translations for each input text
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=100)
        # do we need many samples or just 1?
        outputs = model.generate(**inputs, num_return_sequences=num_samples)
        generated_texts.extend([tokenizer.decode(output, skip_special_tokens=True) for output in outputs])
    return generated_texts

In [ ]:
# Using sentence_bleu to start with. Replace with BleuRT and Comet-QE
# def reward_function(predictions, references):
#     return sentence_bleu(references, predictions)

In [ ]:
# def improve(model, generated_texts, original_texts, tokenizer, batch_size=8, num_epochs=1, learning_rate=5e-5):
#     # Rank and filter the generated texts using the reward function
#     scores = [reward_function(text, original_texts) for text in generated_texts]
#     print(len(scores))
    
#     # Sort the generated texts based on their scores
#     sorted_texts = [x for _, x in sorted(zip(scores, generated_texts), key=lambda pair: pair[0], reverse=True)]
#     print(sorted_texts)
    
#     # Use the top-ranked texts for fine-tuning
#     # For simplicity, let's use the top 50% of the sorted_texts
#     training_data = sorted_texts[:len(sorted_texts) // 2]
#     print(training_data)
#     return None
    
    # # Convert texts to DataLoader for training
    # inputs = tokenizer(training_data, return_tensors="pt", padding=True, truncation=True, max_length=100)
    # dataset = torch.utils.data.TensorDataset(inputs["input_ids"], inputs["attention_mask"])
    # dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # # Define optimizer and scheduler
    # optimizer = AdamW(model.parameters(), lr=learning_rate)
    # scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader) * num_epochs)
    
    # # Fine-tuning loop
    # model.train()
    # for epoch in range(num_epochs):
    #     for batch in dataloader:
    #         input_ids, attention_mask = batch
    #         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
    #         loss = outputs.loss
    #         loss.backward()
    #         optimizer.step()
    #         scheduler.step()
    #         optimizer.zero_grad()
    
    # return model

In [ ]:
# Reinforced Self-Training
def reinforced_self_loop(model, unsupervised_data, supervised_data, optimizer, num_iterations):
    model.train()
    for iteration in range(num_iterations):
        # 1. Translate the unsupervised data using the current model
        inputs = tokenizer(unsupervised_data, return_tensors="pt", padding=True, truncation=True)
        outputs = model.generate(**inputs)
        pseudo_translations = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

        # 2. Compute the reward for the pseudo-translations
        rewards = [reward_function(pred, [ref]) for pred, ref in zip(pseudo_translations, unsupervised_data)]
        # 3. Update the model using the pseudo-translations and their rewards
        # This part is tricky since the M2M100 model isn't directly designed for RL.
        # You'd typically need to define a custom loss function that incorporates the rewards.
        # For simplicity, this step is omitted in this outline.
        # 4. Fine-tune the model on the supervised data"

In [ ]:
#Data Lodaing Function
# Here, you'd typically load your data. For the sake of this example, let's use dummy data:
unsupervised_data = ["This is an unsupervised sentence."] * 10
supervised_data = [("This is a source sentence.", "This is a target sentence.")] * 10

In [ ]:
# load data
import pandas as pd
dev = pd.read_csv('en-ha/cleaned_dev.csv')
train = pd.read_csv('en-ha/cleaned_train.csv')

In [ ]:
train.shape

In [ ]:
# create finetuned model

# Load pre-trained M2M100 model and tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

In [ ]:
# call grow step
# call reward function and score samples
# call improve step
    # for th in threshold
        #  fine-tune model and check performance for improvement
# return model 


In [ ]:
# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Train the model using Reinforced Self-Training
reinforced_self_training(model, unsupervised_data, supervised_data, optimizer, num_iterations=1000)